In [2]:
import numpy as np
import pandas as pd
import json
import os
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import time # To measure time

from scipy.fft import fft
from scipy.signal import detrend, butter, filtfilt
import pywt


from scipy.signal import resample
from collections import Counter


from skimage.transform import resize
from skimage import img_as_float, img_as_ubyte

# Import

In [3]:
# Load subject labels
with open("model-data/Labels_epochs.json", "r") as f:
    subject_labels = json.load(f)

def load_data(directory):
    """
    Loads and returns augmented EEG data (X) and corresponding labels (y) from the specified directory.
    """
    X, y = [], []
    for file in os.listdir(directory):
        if file.endswith(".npy"):
            # Load the features (EEG data)
            X_data = np.load(os.path.join(directory, file))
            # Load corresponding label (subject ID matches file naming convention)
            subject_id = file.split("_")[0]
            label = subject_labels.get(subject_id, None)
            if label is not None:
                X.append(X_data)
                y.append(label)

    return np.array(X), np.array(y)


# Preprocessing

## Model 3

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assume fs (sampling frequency) is known. For example, fs = 250 Hz
fs = 250.0  # Replace this with the actual sampling frequency

# Define your preprocessing function
def preprocess_eeg_data(X, fs, lowcut=1.0, highcut=40.0, order=5):
    """
    Applies preprocessing steps to EEG data:
    - Bandpass filtering
    - Channel-wise standardization (z-score normalization)
    
    Args:
        X (np.ndarray): Raw EEG data (n_subjects, n_epochs, n_channels, n_timesteps).
        fs (float): Original sampling frequency.
        lowcut (float): Lower cutoff frequency for bandpass filter (Hz).
        highcut (float): Upper cutoff frequency for bandpass filter (Hz).
        order (int): Order of the Butterworth filter.

    Returns:
        np.ndarray: Preprocessed EEG data (n_subjects, n_epochs, n_channels, n_timesteps).
    """
    n_subjects, n_epochs, n_channels, n_timesteps = X.shape
    processed_data = np.zeros_like(X)

    # 1. Bandpass Filter Design
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    if low <= 0 or high >= 1:
        print(f"Warning: Filter frequencies ({lowcut}Hz, {highcut}Hz) are invalid for Nyquist freq {nyq}Hz. Skipping filter.")
        b, a = None, None
    else:
        try:
            b, a = butter(order, [low, high], btype='band')
        except ValueError as e:
            print(f"Warning: Could not design Butterworth filter (order={order}, freqs=[{low}, {high}]). Skipping filter. Error: {e}")
            b, a = None, None

    # 2. Apply Filter and Standardize Channel by Channel
    for i_subj in range(n_subjects):
        for i_epoch in range(n_epochs):
            for i_ch in range(n_channels):
                channel_data = X[i_subj, i_epoch, i_ch, :]

                # Apply bandpass filter if designed
                if b is not None and a is not None:
                    try:
                        filtered_data = filtfilt(b, a, channel_data)
                    except Exception as e:
                        print(f"Warning: Filtering failed for subject {i_subj}, epoch {i_epoch}, channel {i_ch}. Using original data. Error: {e}")
                        filtered_data = channel_data
                else:
                    filtered_data = channel_data  # Use original if no filter

                # Standardize (z-score normalization)
                mean = np.mean(filtered_data)
                std = np.std(filtered_data)
                if std > 1e-9:
                    processed_data[i_subj, i_epoch, i_ch, :] = (filtered_data - mean) / std
                else:
                    processed_data[i_subj, i_epoch, i_ch, :] = filtered_data - mean  # Only center if std is zero

    return processed_data




# Models

## Model 3: Rakhmatulin et al., 2024

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EEGNet(nn.Module):
    def __init__(self, n_channels, n_timesteps, num_classes,
                 F1=8, D=2, F2=16, kernel_length=64, dropout_rate=0.5):
        super(EEGNet, self).__init__()
        self.n_channels = n_channels
        self.n_timesteps = n_timesteps
        self.num_classes = num_classes
        self.F1 = F1
        self.D = D
        self.F2 = F2
        self.kernel_length = kernel_length
        self.dropout_rate = dropout_rate

        # Block 1: Temporal Convolution + Depthwise Spatial Convolution
        self.conv1 = nn.Conv2d(19, self.F1, (1, self.kernel_length), padding=(0, self.kernel_length // 2), bias=False)
        self.bn1 = nn.BatchNorm2d(self.F1)

        # Depthwise Conv: applied to each feature map spatially (across channels)
        self.depthwise_conv = nn.Conv2d(self.F1, self.F1 * self.D, (self.n_channels, 1), groups=self.F1, bias=False)
        self.bn2 = nn.BatchNorm2d(self.F1 * self.D)
        
        # Pooling (downsampling)
        self.pool1 = nn.AvgPool2d((1, 4))  # Downsample the time dimension

        # Block 2: Separable Convolution
        separable_kernel_length = 16
        self.separable_conv = nn.Sequential(
            nn.Conv2d(self.F1 * self.D, self.F1 * self.D, (1, separable_kernel_length),
                      padding=(0, separable_kernel_length // 2), groups=self.F1 * self.D, bias=False),
            nn.Conv2d(self.F1 * self.D, self.F2, (1, 1), bias=False)
        )
        self.bn3 = nn.BatchNorm2d(self.F2)
        self.pool2 = nn.AvgPool2d((1, 8))  # Further downsampling

        # Placeholder for fully connected layer, to be created dynamically in forward pass
        self.fc_out = None
        self.dropout = nn.Dropout(self.dropout_rate)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.depthwise_conv(x)
        x = F.elu(self.bn2(x))
        x = self.pool1(x)
        x = self.dropout(x)

        # Block 2
        x = self.separable_conv(x)
        x = F.elu(self.bn3(x))
        x = self.pool2(x)
        x = self.dropout(x)

        # Print the shape before flattening
        print("Shape before flattening:", x.shape)

        # Dynamically calculate the flatten_size
        flatten_size = x.size(1) * x.size(2) * x.size(3)  # Calculate flattened size

        # Create the fully connected layer dynamically with the correct flatten size
        if self.fc_out is None:  # Only initialize once
            self.fc_out = nn.Linear(flatten_size, self.num_classes)

        # Flatten the tensor dynamically
        x = x.view(x.size(0), -1)  # This flattens the tensor

        # Print the shape after flattening
        print("Shape after flattening:", x.shape)

        # Fully connected layer output
        x = self.fc_out(x)
        return x


# Training

In [5]:
# Modify the input data shape before passing to the model in the training loop

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)  # Move model to the designated device
    total_train_steps = len(train_loader)
    if val_loader:
        total_val_steps = len(val_loader)
    start_time = time.time()

    print(f"\n--- Training {model.__class__.__name__} ---")

    for epoch in range(num_epochs):
        # --- Training Phase ---
        model.train()  # Set the model to training mode
        epoch_train_loss = 0.0
        train_correct_predictions = 0
        train_total_samples = 0

        for i, (inputs, labels) in enumerate(train_loader):
            # Move data to the designated device
            inputs = inputs.to(device)  # Shape: (batch_size, 476, 19, 190)
            labels = labels.to(device)

            # Reshape inputs from (batch_size, 476, 19, 190) to (batch_size, 19, 190, 476)
            inputs = inputs.permute(0, 2, 3, 1)  # This changes the shape to (batch_size, 19, 190, 476)
            # Now inputs are (batch_size, 19, 190, 476), which matches the expected input for the model

            # Forward pass
            outputs = model(inputs)  # Model expects input shape (batch_size, 19, 190)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate training statistics
            epoch_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total_samples += labels.size(0)
            train_correct_predictions += (predicted == labels).sum().item()

        # Calculate average training loss and accuracy for the epoch
        avg_epoch_train_loss = epoch_train_loss / total_train_steps
        epoch_train_accuracy = 100 * train_correct_predictions / train_total_samples

        # --- Validation Phase ---
        if val_loader is not None:
            model.eval()  # Set the model to evaluation mode
            epoch_val_loss = 0.0
            val_correct_predictions = 0
            val_total_samples = 0

            with torch.no_grad():  # Disable gradient calculations during validation
                for val_inputs, val_labels in val_loader:
                    # Move data to the designated device
                    val_inputs = val_inputs.to(device)  # Shape: (batch_size, 476, 19, 190)
                    val_labels = val_labels.to(device)

                    # Reshape validation inputs from (batch_size, 476, 19, 190) to (batch_size, 19, 190, 476)
                    val_inputs = val_inputs.permute(0, 2, 3, 1)  # Shape: (batch_size, 19, 190, 476)

                    # Forward pass
                    val_outputs = model(val_inputs)
                    val_loss_batch = criterion(val_outputs, val_labels)

                    # Accumulate validation statistics
                    epoch_val_loss += val_loss_batch.item()
                    _, val_predicted = torch.max(val_outputs.data, 1)
                    val_total_samples += val_labels.size(0)
                    val_correct_predictions += (val_predicted == val_labels).sum().item()

            # Calculate average validation loss and accuracy for the epoch
            avg_epoch_val_loss = epoch_val_loss / total_val_steps
            epoch_val_accuracy = 100 * val_correct_predictions / val_total_samples

            # Print combined epoch results
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                  f'Train Loss: {avg_epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.2f}%, '
                  f'Val Loss: {avg_epoch_val_loss:.4f}, Val Acc: {epoch_val_accuracy:.2f}%')
        else:
            # Print only training results if no validation loader is provided
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                  f'Train Loss: {avg_epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.2f}%')

    end_time = time.time()
    print(f"Finished Training {model.__class__.__name__}. Total time: {end_time - start_time:.2f} seconds")


# Running

## Preprocessing

In [ ]:
# train_data, train_labels = load_data("model-data/epochs-numpy-corrected/train")
# test_data, test_labels = load_data("model-data/epochs-numpy-corrected/test")

# # Check shapes of the loaded data
# print(f"Training data shape: {train_data.shape}")
# print(f"Test data shape: {test_data.shape}")




Training data shape: (240, 476, 19, 190)
Test data shape: (104, 476, 19, 190)


## Model 3

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Load your raw data
train_data, train_labels = load_data("model-data/epochs-numpy-corrected/train")
test_data, test_labels = load_data("model-data/epochs-numpy-corrected/test")

# Preprocess the EEG data
train_data_processed = preprocess_eeg_data(train_data, fs=fs)
test_data_processed = preprocess_eeg_data(test_data, fs=fs)

# Check shapes of the processed data
print(f"Processed training data shape: {train_data_processed.shape}")
print(f"Processed test data shape: {test_data_processed.shape}")


In [ ]:

# Convert processed data to PyTorch tensors
train_data_tensor = torch.tensor(train_data_processed, dtype=torch.float32).to(device)  # Shape: (240, 476, 19, 190)
test_data_tensor = torch.tensor(test_data_processed, dtype=torch.float32).to(device)    # Shape: (104, 476, 19, 190)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder on the training labels
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convert train labels and test labels to PyTorch tensors
train_labels_tensor = torch.tensor(train_labels_encoded, dtype=torch.long).to(device)  # Shape: (240,)
test_labels_tensor = torch.tensor(test_labels_encoded, dtype=torch.long).to(device)   # Shape: (104,)

# Create DataLoader for training and testing
batch_size = 32  # Set the batch size as needed

train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Now you can use the `train_loader` and `test_loader` in your training loop

In [18]:
# Set up the model (assuming you have the number of channels, timesteps, and classes)
model3 = EEGNet(n_channels=19, n_timesteps=190, num_classes=len(set(train_labels)))  # Adjust number of classes
model3.to(device)  # Move model to GPU or CPU

# Loss function (for classification)
criterion3 = nn.CrossEntropyLoss()

# Optimizer (Adam with a learning rate of 0.001)
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)


In [19]:
# Assuming you have train_loader and test_loader set up properly
num_epochs = 1  # Set the number of epochs
train_model(model3, train_loader, test_loader, criterion3, optimizer3, num_epochs, device)



--- Training EEGNet ---
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([16, 16, 172, 15])
Shape after flattening: torch.Size([16, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172

# Evaluation

In [20]:
def evaluate_model(model, data_loader, device):
    """
    Evaluates the model's accuracy on the provided data loader.

    Args:
        model (nn.Module): The trained PyTorch model to evaluate.
        data_loader (DataLoader): DataLoader for the dataset to evaluate on (e.g., test_loader or val_loader).
        device (torch.device): The device to run evaluation on (CPU or CUDA).

    Returns:
        float: The accuracy of the model on the dataset (in percentage).
               Returns 0.0 if the data_loader is empty or None.
    """
    if data_loader is None or len(data_loader) == 0:
        print("Warning: Evaluation data loader is empty or None. Returning 0.0 accuracy.")
        return 0.0

    model.to(device)  # Ensure model is on the correct device
    model.eval()      # Set the model to evaluation mode (disables dropout, uses running means/vars for BatchNorm)

    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculations - crucial for evaluation efficiency and correctness
        for inputs, labels in data_loader:
            # Move data to the designated device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Check if reshaping is necessary (depending on your model's expected input format)
            if inputs.ndimension() == 4:  # If input is in the format (batch_size, channels, time_steps, features)
                # Reshape or permute as necessary, for example, (batch_size, 19, 190, 476) -> (batch_size, 19, 476, 190)
                inputs = inputs.permute(0, 2, 3, 1)  # Adjust this based on your model's expected input format

            # Forward pass
            outputs = model(inputs)

            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability/logit

            # Update total samples and correct predictions count
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    # Calculate final accuracy
    accuracy = 100 * correct_predictions / total_samples
    print(f"Evaluation Accuracy: {accuracy:.2f}%")
    return accuracy


In [21]:
evaluate_model(model3, test_loader, device)

Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([32, 16, 172, 15])
Shape after flattening: torch.Size([32, 41280])
Shape before flattening: torch.Size([8, 16, 172, 15])
Shape after flattening: torch.Size([8, 41280])
Evaluation Accuracy: 43.27%


43.26923076923077